In [ ]:
!wget https://files.consumerfinance.gov/hmda-historic-loan-data/hmda_2017_ut_first-lien-owner-occupied-1-4-family-records_codes.zip
!unzip hmda_2017_ut_first-lien-owner-occupied-1-4-family-records_codes.zip

--2025-05-21 14:52:54--  https://files.consumerfinance.gov/hmda-historic-loan-data/hmda_2017_ut_first-lien-owner-occupied-1-4-family-records_codes.zip
Resolving files.consumerfinance.gov (files.consumerfinance.gov)... 3.163.189.18, 3.163.189.17, 3.163.189.99, ...
Connecting to files.consumerfinance.gov (files.consumerfinance.gov)|3.163.189.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1846487 (1.8M) [binary/octet-stream]
Saving to: ‘hmda_2017_ut_first-lien-owner-occupied-1-4-family-records_codes.zip’

hmda_2017_ut_first- 100%[===================>]   1.76M  --.-KB/s    in 0.06s   

2025-05-21 14:52:54 (31.0 MB/s) - ‘hmda_2017_ut_first-lien-owner-occupied-1-4-family-records_codes.zip’ saved [1846487/1846487]

Archive:  hmda_2017_ut_first-lien-owner-occupied-1-4-family-records_codes.zip
  inflating: hmda_2017_ut_first-lien-owner-occupied-1-4-family-records_codes.csv  


In [ ]:
!pip install imbalanced-learn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [ ]:
data = pd.read_csv('drive/MyDrive/loan_data.csv')

<ipython-input-3-597fe59d17e3>:1: DtypeWarning: Columns (34,36,38,44,46,59,61) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('drive/MyDrive/loan_data.csv')


In [ ]:
data[['action_taken_name', 'action_taken']].value_counts()

,,count
action_taken_name,action_taken,
Loan originated,1,108018
Loan purchased by the institution,6,34218
Application withdrawn by applicant,4,25016
Application denied by financial institution,3,19015
File closed for incompleteness,5,7650
Application approved but not accepted,2,4315
Preapproval request approved but not accepted,8,159
Preapproval request denied by financial institution,7,34


In [ ]:
# for col in data.columns:
#   print(data[col].value_counts())

# here I'm looking at the data and filtering out the columns to keep
data_useful = data[['agency_abbr', 'loan_type_name', 'property_type_name', 'loan_purpose_name', 'owner_occupancy_name', 'loan_amount_000s', 'preapproval_name', 'action_taken_name', 'msamd_name', 'county_name', 'applicant_ethnicity_name', 'co_applicant_ethnicity_name', 'applicant_race_name_1', 'applicant_race_name_2', 'co_applicant_race_name_1', 'co_applicant_race_name_2', 'applicant_sex_name', 'co_applicant_sex_name', 'applicant_income_000s', 'purchaser_type_name', 'denial_reason_name_1', 'denial_reason_name_2', 'denial_reason_name_3', 'rate_spread']]
# do not encode:

In [ ]:
# create an accepted column to generalise action_taken_name
data_useful.loc[:, 'accepted'] = data_useful['action_taken_name'] == 'Loan originated'

# drop incomplete values
filtered_data = data_useful[data_useful['action_taken_name'] != 'Application withdrawn by applicant']
filtered_data = filtered_data[filtered_data['action_taken_name'] != 'File closed for incompleteness']

filtered_data[['action_taken_name', 'accepted']].value_counts()

# for training I only want to keep data with reasons
na_with_reason = filtered_data[filtered_data['accepted'] == False][filtered_data['denial_reason_name_1'].notna()]
accepted = filtered_data[filtered_data['accepted'] == True]

train_data = pd.concat([na_with_reason, accepted])

from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
train_data['applicant_income_000s'] = imp_mean.fit_transform(pd.DataFrame(train_data['applicant_income_000s']))

<ipython-input-54-145d01fdef0c>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  na_with_reason = filtered_data[filtered_data['accepted'] == False][filtered_data['denial_reason_name_1'].notna()]


In [ ]:
# dealing with NaNs
td2 = train_data.drop(columns=['applicant_race_name_2', 'co_applicant_race_name_2', 'msamd_name', 'rate_spread']).dropna(subset=['county_name'])

td2['denial_reason_name_1'] = td2['denial_reason_name_1'].fillna('No reason')
td2['denial_reason_name_2'] = td2['denial_reason_name_2'].fillna('No reason')
td2['denial_reason_name_3'] = td2['denial_reason_name_3'].fillna('No reason')

td2.isna().sum()
td2['accepted'].value_counts()

,count
accepted,
True,107988
False,13796


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

td2_2 = td2.drop(columns=['accepted', 'action_taken_name', 'denial_reason_name_1', 'denial_reason_name_2', 'denial_reason_name_3'])
enc = OneHotEncoder(handle_unknown='ignore').fit(td2_2)
td3 = enc.transform(td2_2)

In [ ]:
from imblearn import FunctionSampler
from imblearn.pipeline import make_pipeline
from imblearn.under_sampling import ClusterCentroids
from sklearn.cluster import MiniBatchKMeans

cc = ClusterCentroids(
    estimator=MiniBatchKMeans(n_init=1, random_state=0), random_state=42
)

X = td3
y = td2['accepted']

X_res, y_res = cc.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)
reg = LinearRegression().fit(X_train, y_train)

In [ ]:
# standardising the data
from sklearn import preprocessing

scaler = preprocessing.StandardScaler(with_mean=False).fit(X_train)
X_train = scaler.transform(X_train)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

pred = reg.predict(X_test)
int_pred = pred.astype(np.int64)
int_pred[int_pred == 2] = 1
int_y = y_test.astype(np.int64)

# accuracy
accuracy = (int_y == int_pred).sum() / len(int_y)

print(classification_report(int_y, int_pred))
confusion_matrix(int_y, int_pred)

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       0.62      1.00      0.76      2784
           1       1.00      0.37      0.54      2735

    accuracy                           0.69      5519
   macro avg       0.54      0.46      0.44      5519
weighted avg       0.81      0.69      0.65      5519



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


array([[   0,    0,    0],
       [   1, 2782,    1],
       [   0, 1715, 1020]])

In [ ]:
from sklearn import model_selection

skf = model_selection.StratifiedKFold(n_splits=10)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
  print(i)
  y_arr = np.array(y)
  reg = LinearRegression().fit(X[train_index], y_arr[train_index])
  pred = reg.predict(X[test_index])
  int_pred = pred.astype(np.int64)
  int_pred[int_pred == 2] = 1
  int_y = y_arr[test_index].astype(np.int64)
  print((int_y == int_pred).sum() / len(int_y))

0


KeyboardInterrupt: 

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)
pred = clf.predict(X_test)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

[[2784    0]
 [ 448 2287]]
              precision    recall  f1-score   support

       False       0.86      1.00      0.93      2784
        True       1.00      0.84      0.91      2735

    accuracy                           0.92      5519
   macro avg       0.93      0.92      0.92      5519
weighted avg       0.93      0.92      0.92      5519

